In [404]:
# Path ORAM 

from collections import defaultdict, deque
from enum import Enum 
from turtle import update
import random
import string

class Node(object): 

	def __init__(self, ary, idx, left, right): 
		self.val = ary
		self.idx = idx
		self.left = left 
		self.right = right
		
	def __repr__(self):
		return f"({self.idx},{self.val},{self.left},{self.right})"


In [405]:
queue = deque()

def buildingPBTLevelOrder_helper(root, nodeidx):
    '''
    helper function that build a perfect binary tree by level order 
    Input: root and node id
    output: root of a tree that is level ordered 
    '''
    nNode = Node([], nodeidx, None, None) 

    if queue: 
        currNode = queue[0]
    
    if root is None:
        root = nNode
    
    elif currNode.left == None:
        currNode.left = nNode
    
    elif currNode.right == None:
        currNode.right = nNode
        queue.popleft()

    queue.append(nNode)
    return root


In [406]:
# constructing a simple binary tree
# actually each of the int in the arrary should be a pointer to a block of size B
#    0    l0
#   /\
#  1  2   l1
#  /\ /\
# 3 4 5 6 l2

def buildingPBTLevelOrder(root, lst):
    for elem in lst:
        root = buildingPBTLevelOrder_helper(root, elem)
    return root
    
rt = buildingPBTLevelOrder(None, range(15))

In [407]:
def buildNodeLevelDict(root, dct, currlevel):
    ''' 
    Build a dictionary that tracks nodes and their associated ids 
    Input: root of a binary tree, dictionary to accumulate the entries of pair (node id, level) 
    Output: the dictionary that has all (node id, level) of the tree
    '''
    if not root:
        return None
    
    dct[root.idx] = currlevel

    nextlevel = currlevel + 1 
    buildNodeLevelDict(root.left, dct, nextlevel)
    buildNodeLevelDict(root.right, dct, nextlevel)

    return dct 

levelDict = buildNodeLevelDict(rt, {}, 0)    

In [408]:
def getHeight(root):
	'''
	Assumption: the given tree is a binary tree 
	Input: the root node of a tree 
	Output: the height of the tree 
	'''
	if not root: 
		return 0 

	leftHeight = getHeight(root.left)
	rightHeight = getHeight(root.right) 

	return max(leftHeight, rightHeight) + 1 

In [409]:
# Global set up 
N = 28 

# Z numbers of blocks within each bucket
Z = 4 
StashInit = []
LEVELS = getHeight(rt) 

random.seed(11)
position = defaultdict(int, {k:random.randrange(0, (pow(2, LEVELS) - 1)) for k in range(1,N+1)})
Operator = ['rd', 'wr']

sorted_pos = defaultdict(list)

for key,val in position.items():
    sorted_pos[val].append(key)
    

In [410]:
print(position)
print(sorted_pos)

defaultdict(<class 'int'>, {1: 7, 2: 13, 3: 8, 4: 13, 5: 14, 6: 12, 7: 7, 8: 7, 9: 8, 10: 13, 11: 9, 12: 3, 13: 2, 14: 12, 15: 8, 16: 7, 17: 10, 18: 9, 19: 12, 20: 2, 21: 1, 22: 7, 23: 4, 24: 2, 25: 1, 26: 8, 27: 12, 28: 14})
defaultdict(<class 'list'>, {7: [1, 7, 8, 16, 22], 13: [2, 4, 10], 8: [3, 9, 15, 26], 14: [5, 28], 12: [6, 14, 19, 27], 9: [11, 18], 3: [12], 2: [13, 20, 24], 10: [17], 1: [21, 25], 4: [23]})


In [411]:
def concatStash(stsh, anotherL):
    '''
    concantenate another list of (block id, data) to the stash
    Input: original stash, and another list 
    Output: new stash with another list concantenated to it
    '''
    for i in anotherL:
        stsh.append(i)
    return None 

In [412]:
def initialize_tree(root, stsh):
    '''
    initialize the tree based on the position map initialization 
    Input: root of a tree and a stash 
    Output: initialized tree, for each node in the tree, it holds Z number of blocks (data, blockid) and the rest of the blocks go into to the stash 
    '''
    if root is None:
        return None 

    data_ = [("", x) for x in sorted_pos[root.idx]]
    data = data_[:Z]
    concatStash(stsh, data_[Z:])
    
    root.val = data 
    initialize_tree(root.left, stsh)
    initialize_tree(root.right, stsh)
    return None

initialize_tree(rt, StashInit)

In [413]:
def getPath(root, NodeId): 
	'''
	Input: root(root) node, and leaf node(NodeId)
	Output: all nodes along the path from leaf node to root node. 
	'''
	if not root:
		return []
	
	if root.idx == NodeId:
		return [(root.idx, root.val)] 		

	leftPath = getPath(root.left, NodeId) 
	rightPath = getPath(root.right, NodeId)
 
	if leftPath:
		return leftPath +  [(root.idx, root.val)]
	if rightPath: 
		return rightPath + [(root.idx, root.val)]
		
	return []

In [414]:
def clearPath(root, leafId): 
    '''
    remove all blocks of nodes along the path from leaf to the root
    Input: root of a tree 
    Output: updated tree with no blocks on the path from given leaf to root
    '''
    if not root: 
        return False  
        
    temp = root.val
    root.val = []
    if root.idx == leafId: 
        return True  
    
    lp = clearPath(root.left, leafId)
    rp  = clearPath(root.right, leafId)
    
    if lp or rp: 
        return True 
    
    root.val = temp
    return False 


In [415]:
def getDataInBlock(root, blockId):
  ''' 
  Get the data associated with a block ID in a given tree
  Input: root of a tree and a block ID
  Output: the data associated with the block ID 
  '''
  if root is not None:
    print("root val is {}".format(root.val))
    for elem in iter(root.val):
      if elem[1] == blockId:
        return elem[0]

    lres = getDataInBlock(root.left, blockId)
    if lres is not None: 
      return lres 

    rres = getDataInBlock(root.right, blockId)
    if rres is not None:
      return rres

In [416]:
def readBlockFromStsh(stsh, blockID):
    '''
    get the value associated with a give block id from the stash 
    Input: a stash that has a collection of pair (data, blockid), and a given block
    Ouput: the value associate with a given blockID 
    '''
    if stsh is None: 
        return None 
    for elem in stsh:
        if elem[1] == blockID:
            return elem[0]

In [417]:
def getNodeAtLevel(root, leafidx, level):
  '''
  get a node at a given level
  Input: the root of a tree, the leaf node index, a specific level
  Output: the node along the path from a given leaf node to root node at the given level 
  '''
  path = getPath(root, leafidx)
  path_len = len(path)
  if level > path_len - 1:
    return None
  return path[path_len-level-1]

In [418]:
def ReadnPopNodes(root, leaf, stsh):
    '''
    read all of the (block, data) pair along the path from a leaf node to the root node
    Input: the root of a tree, a leaf node, and a stash 
    Output: the updated stash and the updated tree
    Thoughts: consider merging getPath() and clearPath() to be the same function since they share the same recursion function. 
    '''
    for l in range(LEVELS):
        nd = getNodeAtLevel(root, leaf, l)
        if nd is not None:
            datum = nd[1] 
            for elem in datum: 
                stsh.append(elem)
  
    clearPath(root, leaf)
    return stsh

In [419]:
def getCandidateBlocksHelper(root, leaf, blockID, level, stsh): 
	'''
	helper function for finding all candidate blocks that satisfy the criteria P(x,l) = P(position[a'],l)
	Input: a root of a binary tree, a leaf node id, a block id, a level, and a stash
	Output: (node.val, node.idx)
	'''
	lhs = getNodeAtLevel(root, leaf, level)
	rhs = getNodeAtLevel(root, position[blockID], level)
	if lhs is not None and rhs is not None:
		if lhs == rhs:
			data = readBlockFromStsh(stsh, blockID)
			res = (data, blockID)
			return res
		else:
			return None 
	return None

In [420]:
def getCandidateBlocks(root, leaf, level, stsh):
    '''
    find all candidate blocks that satisfy the criteria P(x,l) = P(position[a'],l)
    Input: a root of a binary tree, a leaf node id, a level, and a stash 
    Output: all candidate blocks
    '''
    acc = []
    for elem in stsh:
        blocks = getCandidateBlocksHelper(root, leaf, elem[1], level, stsh)
        acc.append(blocks)
    res = [x for x in acc if x is not None]
    return res

In [421]:
def writeBackNodes(root, leafIdx, tgtlevel, data):
    '''
    implementation of WriteBucket(P(x,l), S') in the paper 
    Input: root of a binary tree, leaf id, target level of the write, and new data to write to the node
    Output: None
    '''
    if not root:
        return False 
    
    temp = root.val 
    if levelDict[root.idx] == tgtlevel: 
        root.val = data
    
    if root.idx == leafIdx:
        return True
    
    lb = writeBackNodes(root.left, leafIdx, tgtlevel, data)
    rb = writeBackNodes(root.right, leafIdx, tgtlevel, data)

    if lb or rb: 
        return True

    root.val = temp 
    return False


In [422]:
def updateStash(blockID, dataN, stsh):
    '''
    update the data associated with block id with dataN in the stash 
    Input: block id, new data associated with the block id, and a stash 
    Output: updated stash 
    '''
    blockIndex = -1
    for (idx, pair) in enumerate(stsh):
        temp = list(pair)
        if temp[1] == blockID:
            blockIndex = idx 
    assert blockIndex != -1, "leafid not found in tree"
    stsh[blockIndex] = (dataN, blockID)


In [423]:
def popStash(stsh, items):
    '''
    remove all pairs (data, blockid) from items list
    Input: STASH, and list of pairs to be popped 
    Output: updated STASH 
    '''
    popIds = [] 
    for item in items:
        popIds.append(item[1])
    updatedSTASH = [x for x in stsh if x[1] not in popIds]
    return updatedSTASH

In [424]:
def getBlockIdsFromLst(lst):
    '''
    return all the block ids in a list
    Input: list of pair (data, blockid)
    Output: all block ids in a list
    '''
    return [x[1] for x in lst]

In [425]:
def access(root, opCode, blockId, dataNew=None): 
	'''
	Access algorithm in paper Path ORAM 
	Input: root of a binary tree, operation code, either rd or wr, block id, and if the operation code is wr, dataNew is the new data to be written to the ram, otherwise, left None
	Output: If the operation code is read, there is no output, but if the operation is write, it returns the previous data associated with the block before writing 
	'''
	leafIdx = position[blockId]
	position[blockId] = random.randrange(0, (pow(2, LEVELS) - 1))
	global StashInit
	StashInit = ReadnPopNodes(root, leafIdx, StashInit)

	assert blockId in getBlockIdsFromLst(StashInit),"invariant not upheld!"

	dataOld = readBlockFromStsh(StashInit, blockId)
	if opCode == "wr": 
		updateStash(blockId, dataNew, StashInit)

	for l in reversed(range(LEVELS)):		
		if len(StashInit) != 0:
			candidateBlocks = getCandidateBlocks(root, leafIdx, l, StashInit)
			if (len(candidateBlocks) >= Z):
				writeBackSize = Z
			else:
				writeBackSize = len(candidateBlocks)
		else:
			candidateBlocks = []
			writeBackSize = 0
		writeBackBlocks = candidateBlocks[:writeBackSize] 
		updatedSTASH = popStash(StashInit, writeBackBlocks)
		StashInit = updatedSTASH
		writeBackNodes(root, leafIdx, l, writeBackBlocks)
	return dataOld
	

Development notes:
*1 inspect the STASH 
*2 inspect why there are "None"s in the STASH after initialization
*2 automate the testing 
    - record the sequence (accessIdx, treeRoot, Op, blockId, dataN)
    - parameterized by N 

In [426]:
def validRAM(testN, root):
    for i in range(testN):
        randBlockId = random.randrange(1,N+1)
        dataN = random.choice(string.ascii_letters)
        print("*****************writing access {}******************".format(i))
        dataO = access(root, 'wr', randBlockId, dataN)
        print("*****************reading access {}******************".format(i))
        dataR = access(root, 'rd', randBlockId, )
        print("blockID is {}, dataN is \"{}\", dataR is \"{}\" ".format(randBlockId, dataN,dataR))
        if dataN == dataR:
            print("access # {} True".format(i))
        else:
            print("access # {} False".format(i))

validRAM(60,rt)

*****************writing access 0******************
*****************reading access 0******************
blockID is 23, dataN is "O", dataR is "O" 
access # 0 True
*****************writing access 1******************
*****************reading access 1******************
blockID is 13, dataN is "C", dataR is "C" 
access # 1 True
*****************writing access 2******************
*****************reading access 2******************
blockID is 20, dataN is "P", dataR is "P" 
access # 2 True
*****************writing access 3******************
*****************reading access 3******************
blockID is 1, dataN is "H", dataR is "H" 
access # 3 True
*****************writing access 4******************
*****************reading access 4******************
blockID is 2, dataN is "m", dataR is "m" 
access # 4 True
*****************writing access 5******************
*****************reading access 5******************
blockID is 20, dataN is "b", dataR is "b" 
access # 5 True
*****************writing

assertions 
weakened version of the main invariant: assert it as either the block lives in the path or the stash. 